# Импорт

In [51]:
import sys
sys.path.append('../')

import graphbuilder_v2
import momepy
import folium
import shapely
import osmnx as ox
import pandas as pd
import geopandas as gpd

# Примеры работы

## Загрузка графа

In [52]:
# В качестве начальных данных подается полигон, по которому нужно скачать граф
# lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
# spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
# city = lo_polygon.union(spb_polygon).to_crs(epsg=4326)

city = ox.geocode_to_gdf('R1281563', by_osmid=True)
graph = graphbuilder_v2.get_graph_from_polygon(city, crs=32636)

graph['exit'] = 0
# Преобразуем координатные системы для корректного пересечения
city_transformed = city.to_crs(epsg=32636)
# Находим пересечения ребер с границей города
graph['intersections'] = graph['geometry'].intersection(city_transformed.unary_union)
# Обновляем геометрию ребер
graph['geometry'] = graph['intersections']
# Создаем маску для ребер, пересекающихся с границей города
mask = graph[graph['reg'].isin([1, 2])].intersects(city_transformed.unary_union.boundary)
# Устанавливаем значение 'exit' в 1 для ребер, соответствующих маске
graph.loc[graph['reg'].isin([1, 2]) & mask, 'exit'] = 1
# Убираем временные колонки
graph.drop(columns=['intersections'], inplace=True)

In [67]:
graph[['geometry']]

,geometry
0,None
1,"LINESTRING (1026062.073 5274681.539, 1026060.3..."
2,LINESTRING Z EMPTY
3,LINESTRING Z EMPTY
4,LINESTRING Z EMPTY
...,...
6856,"LINESTRING (1033351.640 5280707.395, 1033317.7..."
6857,"LINESTRING (1033072.998 5280826.924, 1033160.6..."
6858,None
6859,None


In [68]:
graph_filtered = graph[~((graph['geometry'].apply(lambda x: x.is_empty if x else False)) & 
                         (graph['geometry'].apply(lambda x: x.has_z if x else False)))]


graph_filtered[['geometry']]

,geometry
0,None
1,"LINESTRING (1026062.073 5274681.539, 1026060.3..."
2,LINESTRING Z EMPTY
3,LINESTRING Z EMPTY
4,LINESTRING Z EMPTY
...,...
6856,"LINESTRING (1033351.640 5280707.395, 1033317.7..."
6857,"LINESTRING (1033072.998 5280826.924, 1033160.6..."
6858,None
6859,None


In [ ]:
# Конвертация str -> geometry
graph = graphbuilder_v2.convert_geometry_from_wkt(graph)

## Перевод graph -> GeoDataFrame

In [ ]:
# Преобразуем граф в GeoDataFrames
nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)

In [ ]:
import geopandas as gpd
import folium
# edges['exit'] = 0

# intersections = edges['geometry'].intersection(city.unary_union)
# edges['geometry'] = intersections
# edges = edges[~edges['geometry'].is_empty]

# mask = edges.intersects(city.unary_union.boundary)
# edges.loc[mask, 'exit'] = 1
# # Предполагаем, что edges и city уже загружены в виде GeoDataFrame
edges_filtered = edges[edges['reg'] != 3]

# Создаем базовую карту
m = folium.Map()
edges_layer = edges_filtered.explore(m=m, color='blue', name='Edges')
city_layer = city.explore(m=m, color='red', name='Cities')
folium.LayerControl().add_to(m)
m


## Вывод на карту определенный тип reg_status дороги

In [ ]:
edges = edges.to_crs(epsg=4326)
edges_filtered = edges[edges['reg'].isin([1, 2])]

m = folium.Map()

# Определение цветов для разных значений reg
color_map = {1: 'red', 2: 'blue', 3: 'green'}

# Добавление рёбер на карту
for _, row in edges_filtered.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, color=color_map[row['reg']]: {'color': color},
        tooltip=f"highway: {row['highway']}, ref: {row['ref']}, reg: {row['reg']}"
    ).add_to(m)

# Отображение карты
m.save('../data/html/roads.html')

## Проецирование точек на узлы графа

In [ ]:
lomonosov = ox.geocode_to_gdf('N411691832', by_osmid=True).to_crs(epsg=32636)[['geometry','name']]
lomonosov.loc[0, 'name'] = 'Ломоносов'

city_points = gpd.read_file('../data/geojsons/СНП ЛО.geojson')

cities = ["приозерск", "кировск", "кингисепп", "луга", 
          "лодейное поле", "гатчина", "тихвин", "тосно", 
          "выборг", "бокситогорск", "всеволожск", "волосово", 
          "волхов", "сосновый бор", "сланцы", "подпорожье", "кириши"]

# Выделение административных центров
city_points = city_points[
    (city_points['name'].str.lower().isin(cities)) &
    (city_points['rural settlement'].str.contains('административный центр'))
].sort_values('name')

city_points = city_points.to_crs(epsg=32636)
city_points = pd.concat([city_points, lomonosov]).reset_index(drop=True)

nodes_fil = nodes[(nodes['reg_1'] == True) | (nodes['reg_2'] == True)]
graph_2 = graphbuilder_v2.assign_city_names_to_nodes(city_points, nodes_fil, graph, name_attr='city_name', node_id_attr='nodeID', name_col='name', max_distance=1200)